# **Proyecto Statistical Learning**
# Segunda Parte
**José Barrios - 20007192** 

> La primera parte del proyecto se encuentra en el archivo **Training.ipynb**

## Descripción general
El proyecto consiste en hacer una  clasificación binaria para determinar si una persona sobrevive (y=1), o no (y=0), el hundimiento del Titanic.

Está dividido en dos partes. En esta segunda parte nos enfocaremos en obtener los modelos obtenidos de la primera parte (que fue el entrenamiento) y realizar predicciones con ellos, para luego determinar su precisión.

## Datos y Feature Engineering
Obtenemos los datos de test. En la primera parte del proyecto se creó un archivo llamado "test_rows.txt" que contiene los PassengerId de las observaciones que se reservaron específicamente para ser usados acá y no en la primera de entrenamiento.

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn as sk
import sklearn.preprocessing as skp 
from sklearn import tree
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

import datetime

import pickle

In [3]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Instructions for updating:
non-resource variables are not supported in the long term
Enabled compatitility to tf1.x


In [4]:
data = pd.read_csv('data_titanic_proyecto.csv')
data.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N


Acá procederemos a hacer la misma limpieza de data que se hace en la parte de entrenamiento. Esto normalmente no se haría como en este caso, sino que en ambientes de producción ya existan dlls con la funcionalidad para realizar esto.

In [5]:
#Eliminación de columnas que no se usaron en la primera parte
data = data.drop(columns = ["Name", "Ticket", "Cabin"])

#Imputación con la media de la edad
data["Age"] = data["Age"].fillna(data["Age"].mean())

#One-hot-encoding de las features que son clases
data = pd.concat([data, pd.get_dummies(data["Embarked"], prefix = "Embarked")], axis = 1)
data = pd.concat([data, pd.get_dummies(data["passenger_class"], prefix = "Class")], axis = 1)
data = pd.concat([data, pd.get_dummies(data["passenger_sex"], prefix = "Sex")], axis = 1)

data = data.drop(columns = ["Embarked", "passenger_class", "passenger_sex"])

#Convertimos la columna Passenger_survived a 1s y 0s
data["Survived"] = 1
data.loc[data["passenger_survived"] == "N", "Survived"] = 0
data = data.drop(columns = ["passenger_survived"])

data.head()

,PassengerId,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Class_Lower,Class_Middle,Class_Upper,Sex_F,Sex_M,Survived
0,1,22.0,1,0,7.2500,0,0,1,1,0,0,0,1,0
1,2,38.0,1,0,71.2833,1,0,0,0,0,1,1,0,1
2,3,26.0,0,0,7.9250,0,0,1,1,0,0,1,0,1
3,4,35.0,1,0,53.1000,0,0,1,0,0,1,1,0,1
4,5,35.0,0,0,8.0500,0,0,1,1,0,0,0,1,0


Leer archivo con los ids de test y luego crear un dataset con sus observaciones correspondientes.

In [19]:
#Leer archivo y guardar en dataframe
TestIdsFile = open(r"test_rows.txt","r")
TestIds = pd.DataFrame({"Id": TestIdsFile.readlines()})
TestIdsFile.close()

#Quitar los "\n" de cada fila
TestIds["Id"] = TestIds["Id"].str.replace("\n", "")
TestIds = TestIds.astype({"Id" : int})

#Filtrar las observaciones que nos interesan para test
data_test = data[data['PassengerId'].isin(TestIds["Id"])]
data_test.head()

,PassengerId,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Class_Lower,Class_Middle,Class_Upper,Sex_F,Sex_M,Survived
2,3,26.0,0,0,7.9250,0,0,1,1,0,0,1,0,1
3,4,35.0,1,0,53.1000,0,0,1,0,0,1,1,0,1
8,9,27.0,0,2,11.1333,0,0,1,1,0,0,1,0,1
14,15,14.0,0,0,7.8542,0,0,1,1,0,0,1,0,0
16,17,2.0,4,1,29.1250,0,1,0,1,0,0,0,1,0


## Recuperación de modelos
### Decision Tree
Modelo realizado con ScikitLearn, así que utilizamos pickle para su recuperación.

In [20]:
model_tree = pickle.load(open("models/model_tree.pkl", 'rb'))

In [21]:
prediccion_tree = model_tree.predict(skp.scale(data_test[["Age", "SibSp", "Parch", "Fare", "Embarked_C", "Embarked_Q", "Embarked_S", "Class_Lower", "Class_Middle", "Class_Upper", "Sex_F", "Sex_M"]]))

### SVM
Realizado con SK

In [22]:
model_svm = pickle.load(open("models/model_svm.pkl", 'rb'))

In [23]:
prediccion_svm = model_svm.predict(skp.scale(data_test[["Age", "SibSp", "Parch", "Fare", "Embarked_C", "Embarked_Q", "Embarked_S", "Class_Lower", "Class_Middle", "Class_Upper", "Sex_F", "Sex_M"]]))

### Naive Bayes
También realzado con SK

In [24]:
model_bayes = pickle.load(open("models/model_bayes.pkl", 'rb'))

In [25]:
prediccion_bayes = model_bayes.predict(skp.scale(data_test[["Age", "SibSp", "Parch", "Fare", "Embarked_C", "Embarked_Q", "Embarked_S", "Class_Lower", "Class_Middle", "Class_Upper", "Sex_F", "Sex_M"]]))

### Regresión Logística
Hecho aplicando Sigmoid y sus parámetros se almacenaron en un archivo de texto propio, así que procedemos a su recuperación.

In [42]:
#Leer archivo y guardar en dataframe
SigmoidFile = open(r"models/model_sigmoid.txt","r")
SigmoidTheta = pd.DataFrame({"Theta": SigmoidFile.readlines()})
SigmoidFile.close()

#Quitar los "\n" de cada fila
SigmoidTheta["Theta"] = SigmoidTheta["Theta"].str.replace("]\n", "")
SigmoidTheta["Theta"] = SigmoidTheta["Theta"].str.replace("[", "")
SigmoidTheta = SigmoidTheta.astype({"Theta" : "float32"})

SigmoidTheta.head()

,Theta
0,-0.220160
1,-0.223626
2,-0.035711
3,0.226554
4,0.138948


In [43]:
theta = SigmoidTheta.to_numpy()
theta.dtype

dtype('float32')

In [30]:
class SigmoidModel:
    def __init__(self, theta):
        self.theta = theta
    
    def logits(self, x):
        return tf.matmul(x, self.theta)
    
    def prediction(self, x):
        return tf.math.sigmoid(self.logits(x))
    
    def accuracy(self, predictions, labels):
        _, accuracy = tf.metrics.accuracy(labels = tf.argmax(labels, 1),
                                          predictions = tf.argmax(predictions, 1))
        return accuracy

In [31]:
def Sigmoidprediction(theta, x):
    g = tf.Graph()
    
    with g.as_default():
        modelo = SigmoidModel(theta)
        
        tensor_x = tf.placeholder(tf.float32, [None, x.shape[1]], "tensor_x")
        prediction = modelo.prediction(tensor_x)
        
        with tf.train.MonitoredSession() as session:
            parameters_dict = {tensor_x: x}
            data = session.run(prediction, feed_dict = parameters_dict)
            result = np.where(data < 0.5, 0, 1) 
            result = result.reshape(-1, )
            return result

In [35]:
x_test = data_test[["Age", "SibSp", "Parch", "Fare", "Embarked_C", "Embarked_Q", "Embarked_S", "Class_Lower", "Class_Middle", "Class_Upper", "Sex_F", "Sex_M"]]
x_test = skp.scale(x_test)
x_test.shape

(178, 12)

In [44]:
prediccion_sigmoid = Sigmoidprediction(theta, x_test)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


## Ensemble Prediction
Ahora reunimos los resultados de los 4 modelos para determinar su precisión

In [45]:
resultados = pd.DataFrame({
    "Tree": prediccion_tree, 
    "SVM": prediccion_svm, 
    "Bayes": prediccion_bayes, 
    "Sigmoid": prediccion_sigmoid})
resultados.head()

,Tree,SVM,Bayes,Sigmoid
0,0,0,1,1
1,1,1,1,1
2,0,0,1,1
3,0,0,1,1
4,0,0,0,0


In [47]:
#Cuando el promedio de cada fila sea mayor a 0.5, se dirá que sobrevivió
resultados['Survived'] = resultados.mean(numeric_only=True, axis=1)
resultados["Predicted"] = np.where(resultados['Survived'] < 0.5, 0, 1)
resultados = resultados.drop(["Survived"], axis=1)
resultados["Real"] = data_test[["Survived"]].to_numpy()
resultados.head()

,Tree,SVM,Bayes,Sigmoid,Predicted,Real
0,0,0,1,1,1,1
1,1,1,1,1,1,1
2,0,0,1,1,1,1
3,0,0,1,1,1,0
4,0,0,0,0,0,0


In [48]:
print("Precisión de pruebas")
print("Precisión de Tree: ", accuracy_score(resultados["Tree"], resultados["Real"]))
print("Precisión de SVM: ", accuracy_score(resultados["SVM"], resultados["Real"]))
print("Precisión de Bayes: ", accuracy_score(resultados["Bayes"], resultados["Real"]))
print("Precisión de Sigmoid: ", accuracy_score(resultados["Sigmoid"], resultados["Real"]))
print("Precisión de Ensemble: ", accuracy_score(resultados["Predicted"], resultados["Real"]))

Precisión de pruebas
Precisión de Tree:  0.7247191011235955
Precisión de SVM:  0.797752808988764
Precisión de Bayes:  0.8089887640449438
Precisión de Sigmoid:  0.8033707865168539
Precisión de Ensemble:  0.8146067415730337


## Conclusión
* Las librerías que ofrece Python vienen con funciones que permiten exportar e importar modelos. En este proyecto usamos una versión simplificada, pero normalmente ofrece otras opciones de seguridad y versionamiento a la hora de actualizar modelos a disco.
* Se consigió un Ensemble Learner con una precisión de más de 80%, verificando la precición aproximada que tuvimos en la validación cruzada del entrenamiento.